In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1646555837149_0017,pyspark,dead,Link,Link,


In [2]:
%%configure -f
{
    "proxyUser": "user_nmlapid_user",
    "driverMemory": "8000M",
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    },
    "kind": "pyspark"
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1646555837149_0017,pyspark,dead,Link,Link,


In [3]:
spark.sparkContext.getConf().get('spark.driver.memory')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,application_1646555837149_0021,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'8000M'

In [4]:
sc.install_pypi_package("pip==22.0.3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/6a/df/a6ef77a6574781a668791419ffe366c8acd1c3cf4709d210cb53cd5ce1c2/pip-22.0.3-py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1

  Cache entry deserialization failed, entry ignored

In [5]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl (10.4 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)

In [6]:
sc.install_pypi_package("Pillow")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached Pillow-9.0.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)

In [7]:
sc.install_pypi_package("pyarrow")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pyarrow-7.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.5
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1646573745019-0
    Can't uninstall 'numpy'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python37-sagemaker-pyspark 1.4.1 requires pyspark==2.3.4, which is not installed.
tensorflow 2.4.1 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0 which is incompatible.
tensorflow 2.4.1 requires gast==0.3.3, but you have gast 0.5.2 which is incompatible.
tensorflow 2.4.1 requires grpcio~=1.32.0, but you have grpcio 1.41.0 which is incompatible.
tensorflow 2.4.1 req

In [8]:
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import warnings
warnings.filterwarnings('ignore')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# This is just to take a look at the dataset, this dataframe is not really needed
# since the training data will be obtained from the images themselves

# df_masks = spark.read.csv('s3://pguevara-bdcc2022/mask_project/'
#                           'medical-masks-part1/df_masks.csv', header=True)
# df_masks.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Modify the wildcards in the below filepath to control which/how many images are loaded
# into the images dataframe

images = (spark.read.format("binaryFile").
          option("pathGlobFilter", "*.jpg").
          load('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/??????_?_??????_*.jpg'))

images.limit(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, modificationTime: timestamp, length: bigint, content: binary]

In [12]:
images.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------+--------------------+
|                path|   modificationTime|  length|             content|
+--------------------+-------------------+--------+--------------------+
|s3://pguevara-bdc...|2022-02-23 13:55:45|11650005|[FF D8 FF E0 00 1...|
|s3://pguevara-bdc...|2022-02-22 16:34:38|11509391|[FF D8 FF E1 3B 4...|
|s3://pguevara-bdc...|2022-02-22 19:56:52|10428056|[FF D8 FF E1 02 F...|
|s3://pguevara-bdc...|2022-02-23 14:09:33|10409691|[FF D8 FF E1 2E 2...|
|s3://pguevara-bdc...|2022-02-24 09:33:08|10343758|[FF D8 FF E1 2B 3...|
+--------------------+-------------------+--------+--------------------+
only showing top 5 rows

In [13]:
images.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23943

In [14]:
images.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

In [15]:
model = ResNet50(include_top=False)
# model.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
  """
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)


def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Pandas UDFs on large records (e.g., very large images) can run into Out Of Memory (OOM) errors.
# If you hit such errors in the cell below, try reducing the Arrow batch size via `maxRecordsPerBatch`.
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "128")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# We can now run featurization on our entire Spark DataFrame.
# NOTE: This can take a long time (about 10 minutes) since it applies a large model to the full dataset.
features_df = images.repartition(16).select(col("path"), featurize_udf("content").alias("features"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Extracts the target label from the filepath
# Converts the features into a vector

list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df = (features_df.
      withColumn('label', F.substring('path', 71, 1).cast('int')).
      withColumn('features', list_to_vector_udf(F.col('features'))))


df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1646555837149_0021/container_1646555837149_0021_01_000008/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/mnt/yarn/usercache/livy/appcache/application_1646555837149_0021/container_1646555837149_0021_01_000008/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/mnt/yarn/usercache/livy/appcache/application_1646555837149_0021/container_1646555837149_0021_01_000008/pyspark.zip/pyspark/sql/pandas/serializers.py", line 273, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/mnt/yarn/usercache/livy/appcache/application_1646555837149_0021/container_1646555837149_0021_01_000008/pyspark.zip/pyspark/sql/pandas/serializers.py", line 81, in dump_stream
    for ba

In [22]:
# This is just to save a copy of the featurized dataset (before train-test splitting)
# in parquet format on the s3 bucket
# But this may cause errors if the dataframe is too big?
# df.write.mode("overwrite").parquet('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/tmp/dataset')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

In [24]:
df_train, df_test = df.randomSplit([0.75, 0.25])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
rf_class = RandomForestClassifier(featuresCol='features',
                                  labelCol='label',
                                  maxBins=1000, seed=1)

rf_trained = rf_class.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/19/statements/23 with error payload: {"msg":"requirement failed: Session isn't active."}


In [26]:
df_predictions = rf_trained.transform(df_test)
df_predictions.show(5)

VBox()

An error was encountered:
Session 19 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
	at io.netty.channel.nio.NioEventLoop.run(NioEventLoop.java:500)
	at io.netty.util.concurrent.SingleThreadEventExecutor$4.run(SingleThreadEventExecutor.java:989)
	at io.netty.util.internal.ThreadExecutorMap$2.run(ThreadExecutorMap.java:74)
	at io.netty.util.concurrent.FastThreadLocalRunnable.run(FastThreadLocalRunnable.java:30)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.newClosedChannelException(AbstractChannel.java:957)
	... 12 more
22/03/06 13:47:36 INFO ExecutorAllocationManager: Error reaching cluster manager.
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.scheduler.cluster.CoarseGrainedSchedul

In [27]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " +
      str(evaluator.evaluate(df_predictions.select("prediction", "label"))))

VBox()

An error was encountered:
Session 19 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
	at io.netty.channel.nio.NioEventLoop.run(NioEventLoop.java:500)
	at io.netty.util.concurrent.SingleThreadEventExecutor$4.run(SingleThreadEventExecutor.java:989)
	at io.netty.util.internal.ThreadExecutorMap$2.run(ThreadExecutorMap.java:74)
	at io.netty.util.concurrent.FastThreadLocalRunnable.run(FastThreadLocalRunnable.java:30)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.newClosedChannelException(AbstractChannel.java:957)
	... 12 more
22/03/06 13:47:36 INFO ExecutorAllocationManager: Error reaching cluster manager.
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.scheduler.cluster.CoarseGrainedSchedul